### Requirements

In [1]:
%pip install bpy pandas numpy geopandas blender_notebook

Defaulting to user installation because normal site-packages is not writeable
  Using cached geopandas-1.1.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached blender_notebook-0.0.4-py3-none-any.whl.metadata (2.5 kB)
  Using cached pyogrio-0.11.0-cp311-cp311-macosx_12_0_arm64.whl.metadata (5.3 kB)
  Using cached pyproj-3.7.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (31 kB)
  Using cached shapely-2.1.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached notebook-7.4.4-py3-none-any.whl.metadata (10 kB)
  Using cached appnope-0.1.4-py2.py3-none-any.whl.metadata (908 bytes)
  Using cached debugpy-1.8.14-cp311-cp311-macosx_14_0_universal2.whl.metadata (1.3 kB)
  Using cached jupyter_client-8.6.3-py3-none-any.whl.metadata (8.3 kB)
  Using cached jupyter_core-5.8.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl

### Create custom Kernel

In [5]:
!yes|blender_notebook install --blender-exec="/Applications/Blender.app/Contents/MacOS/Blender" --kernel-name="BlenderKernel"


Current python interpreter version is not 3.7!
blender_notebook will link pip packages installed in this interpreter to the 
blender embedded python interpreter. Mismatch in python version might cause
problem launching the jupyter kernel. Are you sure to continue?
 [y/N]: Saving files to /Users/leostrietzel/Library/Jupyter/kernels/BlenderKernel
yes: stdout: Broken pipe


### Load Shapedata of Germany

In [2]:
import geopandas as gpd
gdf = gpd.read_file("data/shapefiles/VG5000_GEM.shp")

### Load Our Dataset

In [3]:
import pandas as pd
DEUTSCHLAND_ATALAS = "data/DeutschlandAtlasData.csv"
da_df = pd.read_csv(DEUTSCHLAND_ATALAS, dtype={"ID": str}, sep=';', decimal=',')
da_df["ID"] = da_df["ID"].astype(str)


### Merge

In [4]:
gdf_merged = gdf.merge(da_df, left_on="AGS", right_on="ID", how="left")

### Compute centriods

In [5]:
# Compute centroids
gdf_merged["centroid_geom"] = gdf_merged.geometry.centroid
# Separate coordinates
gdf_merged["centroid_lon"] = gdf_merged.centroid_geom.x
gdf_merged["centroid_lat"] = gdf_merged.centroid_geom.y


### Extract releant columns for Visualization

In [6]:
relevant_columns = gdf_merged.iloc[:, 25:]
relevant_columns.head()

# move coordinates to the front
front_cols = ['ID','centroid_lon', 'centroid_lat']

# Remaining columns
remaining_cols = [c for c in relevant_columns.columns if c not in front_cols]

# New order
new_order = front_cols + remaining_cols

relevant_columns = relevant_columns[new_order]

# Rename coordinates for clarity
relevant_columns = relevant_columns.rename(columns={
    'centroid_lon': 'longitude',
    'centroid_lat': 'latitude'
})


### Normlize for computation effiecy

In [7]:
# Normalize longitude to a 0-10 scale
relevant_columns['longitude_normalized'] = (
    (relevant_columns['longitude'] - relevant_columns['longitude'].min()) /
    (relevant_columns['longitude'].max() - relevant_columns['longitude'].min()) * 10
)

# Normalize latitude to a 0-10 scale
relevant_columns['latitude_normalized'] = (
    (relevant_columns['latitude'] - relevant_columns['latitude'].min()) /
    (relevant_columns['latitude'].max() - relevant_columns['latitude'].min()) * 10
)

In [8]:
reference_frame = relevant_columns[['ID', 'longitude_normalized', 'latitude_normalized']]
# Save the reference frame to a CSV file
reference_frame.to_csv("data/reference_frame.csv", index=False)

In [9]:
relevant_columns.to_csv("data/VIZ.csv", index=False)

### Initialize Blender connection

In [10]:
import bpy
from IPython.display import display, Image
import numpy as np
import os

def fresh_scene ():
    # Deselect all objects
    bpy.ops.object.select_all(action='DESELECT')
    # Select all objects except cameras
    for obj in bpy.context.scene.objects:
        if obj.type != 'CAMERA':
            obj.select_set (True)
        else:
            obj.select_set (False)
    bpy.ops.object.delete()

    # add light
    bpy.ops.object.light_add(type='SUN')
    sun = bpy.context.object
    sun. location = (0, 0, 0)

    from math import radians
    sun. rotation_euler = (radians (204), radians(-133), radians(-133))
    sun.data.energy = 5

def render_result():
    bpy.ops.render.render()
    bpy.data.images['Render Result'].save_render(filepath="img.png")
    display(Image(filename="img.png"))

bpy.context.scene.render.resolution_x = 500

### Add a Vertex for each Gemeinde

In [11]:
fresh_scene()
df = pd.read_csv("/Users/leostrietzel/Projekte/Codes/bpy-gallery/california_housing/VIZ.csv")
reference_frame = pd.read_csv("/Users/leostrietzel/Projekte/Codes/bpy-gallery/california_housing/reference_frame.csv")

vertices = [(row.longitude_normalized, row.latitude_normalized, 0) for row in reference_frame.itertuples(index=False)]

mesh = bpy.data.meshes.new("NormalizedMesh")
obj = bpy.data.objects.new("GermanyNormalizedObject", mesh)
bpy.context.collection.objects.link(obj)

mesh.from_pydata(vertices, [], [])
mesh.update()


### Define function for appending a Node Tree

In [12]:

def append_node_tree_from_file(source_filepath, node_tree_name):
    """
    Appends a specific Geometry Node tree from a source .blend file
    into the current Blender file.

    Args:
        source_filepath (str): The full path to the .blend file
                                containing the node tree.
        node_tree_name (str): The name of the Geometry Node tree
                              to append (e.g., "MyProceduralBuilding").
    """

    # Validate if the source file exists
    if not os.path.exists(source_filepath):
        print(f"Error: Source .blend file not found at '{source_filepath}'")
        return

    # Construct the path to the data block within the .blend file
    # For node groups, the type is 'NodeTree'
    data_path = os.path.join(source_filepath, "NodeTree", node_tree_name)

    try:
        # Use bpy.ops.wm.append to append the node tree
        # The link=False argument means it will create a local copy (append),
        # rather than linking (which maintains a reference to the original file).
        bpy.ops.wm.append(
            filepath=data_path,
            directory=os.path.join(source_filepath, "NodeTree") + os.sep, # Important: directory needs to end with a separator
            filename=node_tree_name,
            # set_active_collection=False, # Removed: This argument is not recognized in some Blender versions
            link=False # Set to True if you want to link instead of append
        )
        print(f"Successfully appended node tree '{node_tree_name}' from '{source_filepath}'")

    except Exception as e:
        print(f"An error occurred while appending the node tree: {e}")


### append Node Tree

In [13]:
source_blend_file = "data/3d_viz/PreMade.blend"
node_tree_to_append = "geo_house" # The exact name of the node tree in the source file
append_node_tree_from_file(source_blend_file, node_tree_to_append)

Successfully appended node tree 'geo_house' from 'data/3d_viz/PreMade.blend'


### set paramenter for Node Tree

In [14]:
obj.modifiers.new(name="GeometryNodes", type='NODES').node_group = bpy.data.node_groups["geo_house"]
bpy.context.view_layer.objects.active = obj
obj.select_set(True)

### add atrributes

In [17]:
attributes = relevant_columns.copy()
attributes = attributes.drop(columns=['ID', 'longitude_normalized', 'latitude_normalized'])

for col in attributes:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].fillna(0)

obj = bpy.data.objects['GermanyNormalizedObject']
for attr_name in attributes:
    # Get the attribute if it exists, otherwise create a new one
    attr = obj.data.attributes.get(attr_name) or obj.data.attributes.new(
        name=attr_name,
        type='FLOAT',  # Assuming all your relevant_columns are float types
        domain='POINT'
    )
    
    # Set the data for the attribute from the corresponding DataFrame column
    attr.data.foreach_set('value', df[attr_name].to_list())

# Update the object's data after adding all attributes
obj.data.update()

In [ ]:

obj = bpy.data.objects['GermanyNormalizedObject']

# Iterate through each column name in relevant_columns

print(f"Added attributes: {relevant_columns} to {obj.name}")